In [ ]:
close("2024-03-30.parquet")

In [ ]:
import requests
from datetime import date
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

class apartment_import_today:
    def __init__(self):
        self.headers = {"X-RapidAPI-Key": "f70d515e3fmshc7081c7447a5c31p1733e0jsnb81c8ac77ac8",
                                  "X-RapidAPI-Host": "bayut.p.rapidapi.com"}
        self.url = "https://bayut.p.rapidapi.com/properties/list"
        self.querystring = {"locationExternalIDs":"6901", #6901 refers to Downtown Dubai
                            "purpose":"for-sale",
                            "hitsPerPage":"25","page":"0",
                            "lang":"en",
                            "sort":"date-desc",
                            "categoryExternalID":"4",
                            "priceMax":"4000000",
                            "areaMin":"120",
                            "roomsMin":"2","roomsMax":"2"}
        self.result = pd.DataFrame()

        self.loop_pages_in_requests()
        self.save_result()
    
    def loop_pages_in_requests(self):
        '''
        Bayut API is paginated (max 25 properties per page). This loops through pages of API for chosen Query 
        and creates a joined dataframe.
        '''
        nbPages = int(requests.get(url, headers=self.headers, params=self.querystring).json().get("nbPages"))

        for page in range(nbPages):
            self.querystring["page"] = str(page)
            response = requests.get(url, headers=self.headers, params=self.querystring).json()
            df = pd.DataFrame(response.get("hits"))
            # df = pd.json_normalize(response.get("hits"))
            self.result = pd.concat([self.result, df], ignore_index=True)
            break
        

    def save_result(self):
        '''
        The joined API calls are saved as a parquet file
        '''
        self.result["ownerID"] = self.result["ownerID"].astype(str) #Some ownerID are not digits. Will need to research
        self.result = self.result.drop('extraFields', axis=1) #This column is empty and schema cannot guess type

        table = pa.Table.from_pandas(self.result)
        pq.write_table(table, str(date.today())+'.parquet')



In [ ]:
# apartment_import_today()
x = apartment_import_today()

In [ ]:
x.loop_pages_in_requests()

In [ ]:
x.save_result()

In [ ]:
pd.read_parquet('2024-03-30.parquet', engine='pyarrow')

In [ ]:
y = response.json().get("hits")[0]

In [ ]:

url = "https://bayut.p.rapidapi.com/auto-complete"

querystring = {"query":"Downtown",
               "hitsPerPage":"25","page":"0",
               "lang":"en"}

headers = {
	"X-RapidAPI-Key": "f70d515e3fmshc7081c7447a5c31p1733e0jsnb81c8ac77ac8",
	"X-RapidAPI-Host": "bayut.p.rapidapi.com"
}

auto_response = requests.get(url, headers=headers, params=querystring)

In [ ]:
keys = ["id","level","externalID","name"]

for hit in auto_response.json().get("hits"):
    print("\n %s - externalID: %s - levels: %s" %(hit.get("name"),hit.get("externalID"),hit.get("level")))
    for level in hit.get("hierarchy"):
        print(list( map(level.get, keys)))

In [ ]:
for key, value in auto_response.json().items():
    print(key + ':', value)

In [ ]:
for key, value in auto_response.json().get("hits")[3].items():
    print(key + ':', value)

In [ ]:
url = "https://bayut.p.rapidapi.com/properties/list"

querystring = {"locationExternalIDs":"6901",
               "purpose":"for-sale",
               "hitsPerPage":"25","page":"0",
               "lang":"en",
               "sort":"date-desc",
               "categoryExternalID":"4",
               "priceMin":"3000000","priceMax":"4000000",
               "areaMin":"120",
               "roomsMin":"2","roomsMax":"2"}

headers = {
	"X-RapidAPI-Key": "f70d515e3fmshc7081c7447a5c31p1733e0jsnb81c8ac77ac8",
	"X-RapidAPI-Host": "bayut.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [ ]:
for key, value in response.json().items():
    print(key + ':', value)

In [ ]:
for key, value in response.json().get("hits")[6].items():
    print(key + ':', value)

In [ ]:
keys = ["level","externalID","name"]
for hit in response.json().get("hits"):
    print("\n")
    for x in range(2,len(hit.get("location"))):
        item = hit.get("location")[x]
        print(list( map(item.get, keys)))

In [ ]:
x = response.json().get("hits")[0].get("updatedAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
x = response.json().get("hits")[24].get("updatedAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
x = response2.json().get("hits")[0].get("updatedAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
x = response2.json().get("hits")[24].get("createdAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:
for hit in response.json().get("hits"):
    createdAt = str(datetime.fromtimestamp(hit.get("createdAt")).strftime('%Y-%m-%d %H:%M:%S'))
    updatedAt = str(datetime.fromtimestamp(hit.get("updatedAt")).strftime('%Y-%m-%d %H:%M:%S'))
    reactivatedAt = str(datetime.fromtimestamp(hit.get("reactivatedAt")).strftime('%Y-%m-%d %H:%M:%S'))

    print("created: %s - updated: %s - reactivated: %s " %(createdAt, updatedAt, reactivatedAt))

In [ ]:
response.json().get("hits")[2]

In [ ]:
import pandas as pd

df = pd.json_normalize(response.json().get("hits"))

In [ ]:
# df_2 = pd.json_normalize(response.json().get("hits"),
#                          record_path=["location","category"],
#                          meta=[""])

df_2 = pd.json_normalize(response.json().get("hits"),
                         meta=[["location","id"]])